VERI SETI: https://www.tcmb.gov.tr/wps/wcm/connect/TR/TCMB+TR/Main+Menu/Istatistikler/Enflasyon+Verileri/Tuketici+Fiyatlari<br>
EVDS API kullanilmiştir

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import evds as ev
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from arch.unitroot import ADF,PhillipsPerron,KPSS,ZivotAndrews # birim kÃ¶k testi

In [ ]:
from scipy.stats import boxcox

In [ ]:
from statsmodels.graphics.tsaplots import month_plot, plot_acf
import statsmodels.api as sm

In [ ]:
from merlion.models.utils.autosarima_utils import nsdiffs

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from scipy.special import inv_boxcox  # boxcox'i tersine Ã§evirmek iÃ§in

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing

In [ ]:
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch, acorr_breusch_godfrey

In [ ]:
from statsmodels.tsa.x13 import x13_arima_select_order # normal order ve seasonal order yapilarini ortaya Ã§ikartir.
import os

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from scipy.stats import ttest_1samp, jarque_bera
from arch import arch_model

In [ ]:
with open("C:/Users/erens/OneDrive/MasaÃ¼stÃ¼/banalysis/evdsapi.txt") as dosya:
    api = dosya.read()
    
evds = ev.evdsAPI(api)

In [ ]:
tufe = evds.get_data(["TP.FG.J0"],startdate="01-01-2003",enddate="01-02-2025",frequency=5)
tufe.rename(columns={"TP_FG_J0":"TUFE"},inplace=True) # veri seti dÃ¼zenleme

In [ ]:
tarih=pd.date_range("01-01-2003",periods=len(tufe),freq="M") # tarih indekslemek iÃ§in
tufe["Tarih"]=tarih
tufe.set_index("Tarih",inplace=True) # aylik indeksleme yaptik


<br>
df = tufe.resample("Y").mean() # yillik ortalama tÃ¼fe (tÃ¼ketici fiyatlari)<br>
df.index = df.index.year<br>
sns.heatmap(df,annot=True,fmt=".2f",cmap="Reds")<br>
plt.show()<br>
# tufe degerlerinden enflasyon hesaplamasi yapilcak ( veri islenecek )<br>
tufe["Enflasyon"] = tufe.pct_change()*100 # aylik enflasyon<br>
print(tufe)<br>


In [ ]:
enfay = tufe.pct_change() * 100
enfay.dropna(inplace=True)
enfay = pd.DataFrame(enfay)
enfay.rename(columns={"TUFE":"Enf"},inplace=True)


<br>
plt.plot(tufe)<br>
plt.show()<br>


In [ ]:
adf = ADF(tufe,trend="ct")
pp = PhillipsPerron(tufe,trend="ct")
kpss = KPSS(tufe,trend="ct")

rint(adf) # regresyon analizi yapiyor - test statistic - p value ve lags degerleri cikiyor<br>
rint(pp)<br>
rint(kpss)

p value -> h0 testi 1 veya 0 || 0 ise h0 reddedilir duragandir.

yapisal kirilma testi<br>
c - sabit - ct sabit trendi - t direkt trend

rint('yapisal kirilma')

In [ ]:
trend = ["c","ct","t"] # yapisal kirilma olabilir mi ?

In [ ]:
for i in trend:
    z = ZivotAndrews(tufe,trend=i)
    #print(z)

h0 'i reddedemedigimiz icin (duragan bulamadigimiz icin) yapisal kirilma (anlamli) yok. - p value 1 ciktigindan h1 reddedilir.

In [ ]:
tufe_train = tufe.iloc[:36]
tufe_test = tufe.iloc[-36:]
#print(tufe_test)

boxcox

In [ ]:
tufe_train_bx,lm = boxcox(tufe_train["TUFE"])
tufe_train["Box-Cox"] = tufe_train_bx


<br>
print(tufe_train)<br>
# regresyon modelinde deterministik trend anlamli cikti.<br>


- - - -

In [ ]:
tufe_train["Aylar"] = tufe_train.index.month
dummy=pd.get_dummies(tufe_train["Aylar"],prefix="Aylar",dtype=int,drop_first=False) #kukla deiŸişkenlere Ã§evrilerek regresyon modeli oluşturulur. Aylar deiŸeri 2 ise Aylar_2 gibi sÃ¼tun deiŸişkeni oluşturur. Kolaylik saiŸlar.
tufe_train=pd.concat([tufe_train,dummy],axis=1)

######## stats models

In [ ]:
x = tufe_train.drop(columns=["TUFE","Box-Cox","Aylar"])
y = tufe_train["TUFE"]
model = sm.OLS(y,x).fit() # stat models regression linear model OLS
# print(model.summary())
# mevsimsellik bir deterministik yok,

######## stokastik mevsimsellik kontrol - merlion

In [ ]:
a = nsdiffs(tufe_train["Box-Cox"],m=12,test="ch")
b = nsdiffs(tufe_train["Box-Cox"],m=12,test="ocsb")

rint(a)<br>
rint(b)

######## indeksleme | regresyon modelleme yapabilmek iÃ§in

In [ ]:
indeks = np.arange(1,len(tufe_train)+1)
tufe_train["T"] = indeks

In [ ]:
x = sm.add_constant(tufe_train["T"])
y = tufe_train["Box-Cox"]

model testing

In [ ]:
model = sm.OLS(y,x).fit()
# print(model.summary())

In [ ]:
pol = PolynomialFeatures(degree=3)
xp = pol.fit_transform(tufe_train["T"].values.reshape(-1,1))

model testing

In [ ]:
model = sm.OLS(y,xp).fit()
# print(model.summary())
# Durbin Watson deiŸeri artmasina raiŸmen p deiŸeri artti

In [ ]:
girdi = np.arange(len(tufe_train.index)+1,len(tufe_train.index)+37)
tahmin = model.predict(pol.fit_transform(girdi.reshape(-1, 1)))
tahmin = inv_boxcox(tahmin,lm) # yukarda kullandigimiz lambda degerini gonderdik
tufe_test["Tahmin"] = tahmin   # duraiŸan olmadiiŸi iÃ§in sahte regresyonlarla Ã§ok verimli işlem yapamiyoruz.
# print(tufe_test) # TUFE deiŸerleriyle Tahmin deiŸerlerimiz arasinda ciddi fark var - anlamsiz.

print(r2_score(tufe_test["TUFE"],tufe_test["Tahmin"]))  # r2 score deiŸeri -5.23 geldi<br>
eiŸitimde %99 luk basari olmasina ragmen testlerde durum kÃ¶tÃ¼. <br>
duraiŸanlik olduiŸunu gÃ¶steriyor.<br>
r2 yapisi sahte regresyon oldugunu anlayabiliyoruz.

########################## USTEL DUZELTMELER

### HOLT MODEL<br>
HOLT model RMSE = 1468.83

In [ ]:
model = Holt(tufe_train["Box-Cox"]).fit(optimized=True) 
tahmin = model.forecast(36) # 36 test verisi
tahmin = inv_boxcox(tahmin,lm) # boxcox dÃ¶nÃ¼şÃ¼mÃ¼

print(r2_score(tufe_test["TUFE"],tahmin))  # -5.2680<br>
print(np.sqrt(mean_squared_error(tufe_test["TUFE"],tahmin))) # HOLT model RMSE = 1468.83

### HOLT WINTERS MODEL<br>
HOLT WINTERS model RMSE min 1471 Ã§ikti.

Ã§arpimsal ve toplamsal olarak tiplere bakilacak<br>
seride 0 deiŸer varsa sifirlayacaiŸindan normalde sikinti Ã§ikartabilen bir model.

In [ ]:
trend_tip = ["add","mul"]
seas_tip = ["add","mul"]
per = range(2,13) # periyot - mevsimsellik kalibi

In [ ]:
sonuc = pd.DataFrame(columns=["Trend","Mevsimsellik","Periyod","R2","RMSE"])

In [ ]:
for i in trend_tip:
    for j in seas_tip:
        for k in per:
            model = ExponentialSmoothing(tufe_train["Box-Cox"],trend=i,seasonal=j,seasonal_periods=k).fit(optimized=True)
            tahmin = model.forecast(36) # 36 veri iÃ§in
            tahmin = inv_boxcox(tahmin,lm)
            rmse = np.sqrt(mean_squared_error(tufe_test["TUFE"],tahmin))
            sonuc = sonuc._append({"Trend":i,"Mevsimsellik":j,"Periyod":k,"RMSE:":rmse},ignore_index=True)

In [ ]:
sonuc = sonuc.sort_values(by="RMSE") # sutun adina gÃ¶re df'yi sortla
# print(sonuc)  minimum RMSE 1471 Ã§ikti.

### ARIMA

In [ ]:
model = auto_arima(tufe_train["Box-Cox"],seasonal=False,trace=False)
# print(model.summary())
# AIC -> -130
# SARIMAX(2, 1, 1) kullanildi

seasonal yapisi kullanilarak

In [ ]:
sonuc = pd.DataFrame(columns=["m","AIC"])

In [ ]:
for i in range(2,13):
    model = auto_arima(tufe_train["Box-Cox"],seasonal=True,trace=False,m=i)
    sonuc = sonuc._append({"m":i,"AIC":model.aic()},ignore_index=True)

In [ ]:
sonuc = sonuc.sort_values(by="AIC")
# print(sonuc)

### SARIMAX

In [ ]:
model = SARIMAX(tufe_train["Box-Cox"],order=(0,1,1),seasonal_order=(2,0,1,12),trend="c").fit()
# ma.l1 0 ise problem yok, 0'dan buyuk degerler icin problemdir
# order parametresinde buna dikkat edicez
# model summary kisminda Model:SARIMAX(0,1,2)x(2,..) gibi devam eden kisimda
# L1'de sikinti olmadiiŸi iÃ§in SARIMAX'in 0,1 degerlerini aldik.
# 2'yi 1'e indirdik ma.l2 1'den buyuk sikintili oldugu icin
# z ve P>|z| degerlerine bakarak cevap verdik.
# intercept yapisinda oldugundan trend="c" (constant-sabit) parametresi ekledik

print(model.summary())<br>
AIC -> -120<br>
SARIMAX(0, 1, 1)

############### ARIMA KULLANILACAK<br>
oto korelasyon olmayacak<br>
arch etkisi olmayacak

q model

In [ ]:
q = acorr_ljungbox(model.resid)
h = het_arch(model.resid,nlags=1,ddof=3)[1] # modeli eiŸitirken 3 parametre kullandiiŸimiz iÃ§in
# serbestlik derecesi ddof 3 oldu.

print(q)<br>
print(h)

##### seasonal order kullanimi

model = SARIMAX(tufe_train["Box-Cox"],order = (0,1,1), seasonal_order=(1,0,1,12),trend="n").fit()

path = r'C:\WinX13\x13as'<br>
arima = x13_arima_select_order(tufe_train["Box-Cox"],x12path = path, outlier=True)<br>
outlier - aykiri gÃ¶zlem varsa dikkate alir

print(arima.order) # order (1,1,0)<br>
print(arima.sorder)  # seasonal order (0,1,1,12)

#######

In [ ]:
enfay = tufe["TUFE"].pct_change() * 100
enfay.dropna(inplace=True)
enfay = pd.DataFrame(enfay)
enfay.rename(columns={"TUFE":"Enf"},inplace = True)

In [ ]:
enf_train = enfay.iloc[:-36]
enf_test = enfay.iloc[-36:]

In [ ]:
ayris = seasonal_decompose(enf_train["Enf"],model="add", period = 12, extrapolate_trend = "freq")
# ayris.plot()
# plt.show()

In [ ]:
veri = pd.concat([
    ayris.observed,
    ayris.trend,
    ayris.seasonal,
    ayris.observed - ayris.seasonal, # toplamsal model kullandigimiz icin / yerine Ã§ikartma yaptik
    ayris.resid],axis=1)        # resid -> rassal

In [ ]:
veri.columns = ["Enf","Trend","Mevsimsellik","Mevsimsel DÃ¼zeltme","Rassal"]

y = mevsimsellik + rassal + trend -> Enflasyon

plt.plot(veri["Mevsimsel DÃ¼zeltme"])<br>
plt.show()

mevsimsel dÃ¼zeltmenin doiŸrusallikla bir korelasyonu var mi ona bakicaz

In [ ]:
indeks = np.arange(1,len(veri)+1) # dogrusallik yapisi (girdi olarak kullanilacakti - linear regression)
# print(indeks)
# plt.plot(indeks)
# plt.show()

yukari yÃ¶nlÃ¼yse (hatasi dÃ¼şÃ¼kse) yÃ¼ksek r'2 deiŸeri Ã§ikar ve doiŸrusal korelasyon vardir.<br>
numpy korelasyon matrisini kullanalim

print(np.corrcoef(veri["Mevsimsel DÃ¼zeltme"],indeks))<br>
0.21 'lik korelasyon Ã§ikti. -(linear deiŸil)<br>
verimizde deterministik bir mevsimsellik var<br>
mevsimselliiŸi dişlayip mevsimsel dÃ¼zeltme katsayisina ulaştik.<br>
mevsimsel dÃ¼zeltmeyi doiŸru modelleyebilirsek deterministik olduiŸu iÃ§in ekleme yapip tahmin sonucu elde edebiliriz.<br>
mevsimsellik yapisinda trend yapisi olmadiiŸi iÃ§in doiŸrusal model yapisi kuramadik - kuramayiz.

mevsimsel olmayan arima modeliyle devam edicez

In [ ]:
model = auto_arima(veri["Mevsimsel DÃ¼zeltme"],seasonal = False, Trace = False)
# print(model.summary()) 
# AIC 458
# arch etkisi - (geriye kalan artiklar Ã¼zerinden kontrol edicez) ve otokorelasyon var mi ?

### oto korelasyon<br>
ac = acorr_ljungbox(model.resid()) # rassal deiŸerler

# arch etkisi<br>
h = het_arch(model.resid(), nlags = 1 , ddof=3)[1] # serbestlik derecesi ddof = 3 ( p + d )

print(ac)  # anlamli korelasyon yok<br>
print(h) # 0.001 arch etkisi var.

In [ ]:
enf_test["Mevsimsellik"] = veri["Mevsimsellik"].iloc[-36:].values
tahmin = model.predict(36)
enf_test["Tahmin"] = tahmin + enf_test["Mevsimsellik"]
# print(enf_test)
# enflasyonla tahmin deiŸerleri arasinda Ã§ok fark var
# bunun nedeni arch etkisinden dolayi,

 mevsimsel dÃ¼zeltmeyi kullanmicaz, sarima kullanicaz.


<br>
for i in range(2,13):<br>
    model = auto_arima(veri["Enf"],seasonal=True,m=i,trace=False)<br>
    print("m:{} Aic: {}".format(i,model.aic()))<br>



<br>
m:2 Aic: 515.2326035115791<br>
m:3 Aic: 509.77785147828354<br>
m:4 Aic: 502.2720704046204<br>
m:5 Aic: 541.3718499213408<br>
m:6 Aic: 509.2182209465135<br>
m:7 Aic: 541.3718499213408<br>
m:8 Aic: 518.7257272095954<br>
m:9 Aic: 528.3586976084922<br>
m:10 Aic: 541.3718499213408<br>
m:11 Aic: 541.3718499213408<br>
m:12 Aic: 494.3792266831782<br>
Bu modelde Aic deiŸeri Ã¶ncekine gÃ¶re yÃ¼ksek Ã§ikti, kullanmayacaiŸiz.<br>


Yumuşatma Ãœstel Modelleri Deneyelim:

In [ ]:
    
sonuc = pd.DataFrame(columns=["DÃ¶nem", "RMSE"])
period = range(2,13)

hold winters mevsimselliiŸi de baz aliyor

In [ ]:
for i in period:
    model = ExponentialSmoothing(enf_train["Enf"],trend="add",seasonal="add",seasonal_periods=i).fit(optimized=True)
    tahmin = model.forecast(36) # 36 dÃ¶nemlik
    rmse = np.sqrt(mean_squared_error(enf_test["Enf"], tahmin))
    sonuc = sonuc._append({"DÃ¶nem":i,"RMSE":rmse},ignore_index=True)
    
    
sonuc = sonuc.sort_values(by="RMSE") # kucukten buyuge
# print(sonuc)


<br>
    DÃ¶nem      RMSE<br>
1     3.0  3.904438<br>
7     9.0  3.912075<br>
0     2.0  3.933386<br>
8    10.0  3.934485<br>
6     8.0  3.948194<br>
9    11.0  3.949118<br>
10   12.0  3.970932<br>
4     6.0  3.997553<br>
2     4.0  4.140709<br>
3     5.0  4.206096<br>
5     7.0  4.227467<br>
Periyot olarak 3 dÃ¶nem kullandiiŸimizda en dÃ¼şÃ¼k RMSE deiŸerine ulaşabiliyoruz<br>
Deneyelim:<br>


In [ ]:
model = ExponentialSmoothing(enf_train["Enf"],trend="add",seasonal="add",seasonal_periods=3).fit(optimized=True)
tahmin = model.forecast(36) # 36 dÃ¶nemlik
enf_test["Tahmin"] = tahmin
# print(enf_test)
    
    
sonuc = sonuc.sort_values(by="RMSE")
# print(sonuc)


<br>
DÃ¶nem      RMSE<br>
0     3.0  3.904438<br>
19   10.0  3.904438<br>
18    9.0  3.904438<br>
17    8.0  3.904438<br>
16    7.0  3.904438<br>
15    6.0  3.904438<br>
14    5.0  3.904438<br>
13    4.0  3.904438<br>
12    3.0  3.904438<br>
11    2.0  3.904438<br>
21   12.0  3.904438<br>
20   11.0  3.904438<br>
1     9.0  3.912075<br>
2     2.0  3.933386<br>
3    10.0  3.934485<br>
4     8.0  3.948194<br>
5    11.0  3.949118<br>
6    12.0  3.970932<br>
7     6.0  3.997553<br>
8     4.0  4.140709<br>
9     5.0  4.206096<br>
10    7.0  4.227467<br>


mevsimsel dÃ¼zeltme = trend + rassal olduiŸundan rassal'lik enflasyona ciddi etkisi var.<br>
sabir bir ortalama modeliyle tahmin etmek zordur<br>
-----------------

# veri gÃ¶rselleştirelim

In [ ]:
veri.plot(y=["Enf","Trend","Rassal"])
#plt.show()

enflasyon rassal şoklarla hareket ediyor.<br>
trend veya rassali tek başina modelleyebilir miyiz ?

trend -> belirli bir ortalama etrafinda gidiyor - linear regresyonla aÃ§iklanmaz<br>
mevsimsellik -> deterministiktir.<br>
rassallik -> bunun Ã¼zerinde durucaz.

In [ ]:
indeks = np.arange(1,len(veri)+1)
veri["T"] = indeks

polinomal regresyonla

In [ ]:
pol = PolynomialFeatures(degree = 3)
xp = pol.fit_transform(veri["T"].values.reshape(-1,1))

In [ ]:
y = veri["Trend"]
model = sm.OLS(y,xp).fit()
# print(model.summary())

Log - Likehood = Katsayilar Anlamli<br>
Prob (F-statistic) > 0 : Model Anlamli<br>
R^2 dÃ¼şÃ¼k Ã§Ã¼nkÃ¼ doiŸrusal model deiŸil.

In [ ]:
tahmin = model.predict(xp)

sns.scatterplot(x = veri["T"], y=veri["Trend"], label="GerÃ§ek DeiŸerler")<br>
sns.lineplot(x = veri["T"], y=tahmin, label="Tahmin")<br>
plt.show()

rint(model.summary()) # polinom derecesi 1 'ken R^2 0.35

In [ ]:
                       # polinom derecesi 2 'ken R^2 0.595
                       # polinom derecesi 3 'ken R^2 0.630

 modelin artiklari

In [ ]:
artik = model.resid

 plot acf - otokoralasyon yapisi<br>
lot_acf(artik, zero=False)<br>
lt.show()

acorr_breusch_godfrey - artiklarin otokoralasyonunu test edicek

In [ ]:
h = acorr_breusch_godfrey(model)[1] # h1 - otokoralasyonun var olduiŸunu | h
# prop deiŸeri Ã¶nemli olduiŸundan modelin [1] deiŸerini aldik.

print(h) - 5.5162208818113e-40  - h1 kabul (h>0)

####### ARIMA ile Trendi tahmin etme<br>
trend modeli

In [ ]:
model = auto_arima(veri["Trend"],seasonal = False, trace= False)
# print(model.summary())
# ar deiŸerleri yok, (mevsimsellik kontrol edilmedi)

In [ ]:
artik = model.resid()
ac = acorr_ljungbox(artik)
# print(ac) # otokoralasyon testi - anlamli deiŸil.

arch etkisi testi

In [ ]:
h = het_arch(artik, nlags=1, ddof=3)[1]
# print(h) 0.9953081082332432 - p-deiŸeri > 0.05 || 0.99 > 0.05 H0 kabul (arch etkisi yok)

## tahmin

In [ ]:
tahmin = model.predict_in_sample()
# print('TAHMIN:')
# print(tahmin)

plt.plot(veri["Trend"],label="Trend")<br>
plt.plot(tahmin,label = "Tahmin")<br>
plt.legend()<br>
plt.show()

##### Rassal Hareket Analizi<br>
 Ozetle<br>
Deterministik Mevsimsellik var<br>
Trend -> arch etkisi yok , otokorelasyon yapisi yok.<br>
Rassal'in Enflasyona etkisi en fazla derecede.<br>
ttest ile

rassalliiŸin ortalamasi sifir mi fark li mi ?

In [ ]:
t_ist,p = ttest_1samp(veri["Rassal"], popmean=0)
# popmean -> hangi degerden farkli olup olmadigini test etmek icin verdigimiz deger
# print(p)
# 0.9255763888159445   -   h0 kabul arima yerine zero modeli secilebilir ( 0 ortalamaya sahip )

## ADF augmented dickey fuller Testi - Rassal<br>
ortalama 0 oldugu icin trend n gonderebiliriz

In [ ]:
adf = ADF(veri["Rassal"],trend="n")
# print(adf)


<br>
Test Statistic: -8<br>
Trend: No Trend<br>
Critical Values: -2.58 (1%), -1.94 (5%), -1.62 (10%)<br>
Null Hypothesis: The process contains a unit root.<br>


## PhillipsPerron

In [ ]:
pp = PhillipsPerron(veri["Rassal"],trend="n")
## print(pp)


<br>
Test Statistic                -16.056<br>
P-value                         0.000<br>
Lags                               15<br>
-------------------------------------<br>
Trend: No Trend<br>
Critical Values: -2.58 (1%), -1.94 (5%), -1.62 (10%)<br>
Null Hypothesis: The process contains a unit root.<br>


#

In [ ]:
sns.histplot(veri["Rassal"],kde=True)
# plt.show()
# kde - normal dagilim

### normallik testi

In [ ]:
n = jarque_bera(veri["Rassal"]) # normal dagilima uymamasini bekliyoruz..
# h0 normal dagilir
# h1 normal dagilmaz

print(n)<br>
SignificanceResult(statistic=1076.5207353062297, pvalue=1.723822325527858e-234)<br>
normal dagilim yok<br>
seri duragan

## ARCH ile modelleyecegiz

In [ ]:
p_val = range(1,5) # 5 i kapsamayacagi icin max 4 gecikme
q_val = range(1,5)
o_val = range(0,2)

In [ ]:
dag = ["ged","normal","studentst","skewt"]
model = ["GARCH","EGARCH"]

In [ ]:
sonuc = pd.DataFrame(columns=["p","o","q","Model","DaiŸilim","Aic"])

In [ ]:
for p in p_val:
    for q in q_val:
        for o in o_val:
            for d in dag:
                for m in model:
                    modelarch = arch_model(veri["Rassal"],p=p,o=o,q=q,vol=m,dist=d,mean="Zero").fit(disp="off")
                    sonuc = sonuc._append({"p":p,"o":o,"q":q,"Model":m,"DaiŸilim":d,"Aic":modelarch.aic},ignore_index=True)

In [ ]:
sonuc = sonuc.sort_values(by="Aic")

print(sonuc)


<br>
p  o  q   Model DaiŸilim         Aic<br>
14   1  1  1   GARCH   skewt  381.491804<br>
30   1  1  2   GARCH   skewt  383.491803<br>
78   2  1  1   GARCH   skewt  383.491804<br>
142  3  1  1   GARCH   skewt  385.491803<br>
46   1  1  3   GARCH   skewt  385.491803<br>
..  .. .. ..     ...     ...         ...<br>
114  2  0  4   GARCH  normal  413.718610<br>
35   1  0  3  EGARCH  normal  414.308609<br>
99   2  0  3  EGARCH  normal  415.158920<br>
51   1  0  4  EGARCH  normal  416.308608<br>
115  2  0  4  EGARCH  normal  417.158921<br>
<br>
model-1 en dÃ¼sÃ¼k AIC deiŸerli;

In [ ]:
modelarch = arch_model(veri["Rassal"],p=1,o=1,q=1,vol="GARCH",dist="skewt",mean="Zero").fit(disp="off")
# print(modelarch)
# ortalamasi zaten sifir

arch etkisi kontrol 

In [ ]:
h = het_arch(modelarch.std_resid, nlags=1)[1]
print(h)
# 0.859623524612982 -> H0 kabul arch etkisi yok.

In [ ]:
plt.plot(veri["Rassal"])
plt.plot(modelarch.conditional_volatility) # aykiri deiŸerleri ne kadar tahmin edebiliyoruz?

### OZET ####<br>
Mevsimsellik -> Deterministik<br>
Trend -> Arima ile modellenecek

##### Rolling Window - Kayan Yapi

In [ ]:
test_sayisi = len(enf_test)
vol_tahmin = [] # volatility olmadigi icin.

In [ ]:
for i in range(test_sayisi):
    train = enfay["Enf"][:-(test_sayisi-i)]
    modelarch = arch_model(train, p=1, o=1, q=1, vol="GARCH", dist="skewt", mean="Zero")
    model_arch = modelarch.fit(disp="off")
    tahmin = model_arch.forecast(horizon=1) # horizon -> kaÃ§ adim sonrasini tahmin edecek??
    vol_tahmin.append(np.sqrt(tahmin.variance.values[-1,:][0])) # Array Yapisindan Ã§ikarmak iÃ§in ekstradan [0] parametresi ekledik.

## Trend Tahmin

In [ ]:
model_trend = auto_arima(veri["Trend"],seasonal=False, trace=False)
tahmin_trend = model_trend.predict(test_sayisi)

In [ ]:
tahmin_mev = veri["Mevsimsellik"].iloc[-(test_sayisi):]
# print(tahmin_mev)

# sonuc - toplamsal model

In [ ]:
plt.figure()
son = tahmin_trend + tahmin_mev.values
enf_test["Tahmin"] = son

In [ ]:
plt.ylim(top=13)
plt.plot(enf_train["Enf"], label = "Train")
plt.plot(enf_test["Enf"], label="Test")
plt.plot(enf_test["Tahmin"], label="Tahmin")
plt.legend()
plt.show()

şoklari (volatility) tahmin edemiyor.<br>
eklersek (volatility degerlerini)

In [ ]:
plt.figure()
son = tahmin_trend + tahmin_mev.values + vol_tahmin
enf_test["Tahmin"] = son
plt.ylim(top=13)
plt.plot(enf_train["Enf"], label = "Train")
plt.plot(enf_test["Enf"], label="Test")
plt.plot(enf_test["Tahmin"], label="Tahmin")
plt.legend()
plt.show()